In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
!wget https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar
!tar -xf abo-images-small.tar

In [2]:
import torch
# ========= CONFIG =========
listing = "0"
csv_path = "/kaggle/input/abo-dataset-merged/merged_listings.csv"  # CSV must contain: image_path, question, one_word_answer
#get length of csv filevvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
f = open(csv_path, "r")
lines = f.readlines()
f.close()


model_dir = "/kaggle/working/models"
output_dir = "/kaggle/working/lora_on_listing"
images_root = "/kaggle/input/abo-dataset"
batch_size = 4
lenght_of_dataset = len(lines) -1
max_steps = (len(lines)-1)//batch_size
print(f"Steps per epoch: {max_steps}")
checkpoint_path = None
# ===========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
del lines

Steps per epoch: 304298


### Unsloth

In [24]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch
from transformers import AutoProcessor




model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)



==((====))==  Unsloth 2025.5.2: Fast Qwen2_Vl patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [25]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 4,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 4,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

Unsloth: Making `model.base_model.model.visual` require gradients


In [26]:
from torch.utils.data import IterableDataset, DataLoader
from PIL import Image
class VQALazyIterableDataset(IterableDataset):
    def __init__(self, dataset_stream, lenght_of_dataset,tokenizer):
        self.dataset_stream = dataset_stream
        self.lenght_of_dataset = lenght_of_dataset
        self.images_root = "/kaggle/input/abo-dataset/abo_small/small"
        self.tokenizer = tokenizer

    def preprocess(self, example):


        image = Image.open(f"{self.images_root}/{example['image_path']}").convert("RGB")
        image = image.resize((224, 224))
        question = example["question"]
        answer = example["answer"]
        if not answer 

        messages = [
            {"role": "user", "content": [
                {"type": "image"},
                {"type": "text", "text": question}
            ]}
        ]
        input_text = self.tokenizer.apply_chat_template(messages, add_generation_prompt = True)
        inputs = self.tokenizer(
            image,
            input_text,
            add_special_tokens = False,
            max_length=300,
            truncation=True,
            return_tensors = "pt",
            padding="max_length"
        )

        labels = self.tokenizer.tokenizer(
                text=answer,
                add_special_tokens = False,
                max_length=300,
                truncation=True,
                padding="max_length",
                return_tensors = "pt",
            ).input_ids



        return {
            "pixel_values": inputs["pixel_values"].squeeze(0),
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": labels.squeeze(0),
            "image_grid_thw": inputs["image_grid_thw"].squeeze(0)
            }



    def __iter__(self):
        for example in self.dataset_stream:
            processed = self.preprocess(example)
            if processed:
                yield processed
    def __len__(self):
      return self.lenght_of_dataset



In [27]:
def collate_fn(batch):
    import torch

    # Extract fields from batch
    pixel_values = [example["pixel_values"] for example in batch]
    input_ids = [example["input_ids"] for example in batch]
    attention_mask = [example["attention_mask"] for example in batch]
    labels = [example["labels"] for example in batch]
    image_grid_thw = [example["image_grid_thw"] for example in batch]

    # Stack vision-related tensors
    pixel_values = torch.stack(pixel_values)         # [B, 256, 1176]
    image_grid_thw = torch.stack(image_grid_thw)     # [B, 3]

    # Tokenizer padding for input_ids & attention_mask
    text_inputs = [
        {"input_ids": ids, "attention_mask": mask}
        for ids, mask in zip(input_ids, attention_mask)
    ]
    padded_inputs = tokenizer.tokenizer.pad(
        text_inputs,
        return_tensors="pt"
    )

    # Tokenizer padding for labels
    padded_labels = tokenizer.tokenizer.pad(
        [{"input_ids": lbl} for lbl in labels],
        return_tensors="pt"
    )["input_ids"]

    # Replace padding token id with -100 for loss masking
    padded_labels[padded_labels == tokenizer.tokenizer.pad_token_id] = -100

    return {
        "pixel_values": pixel_values,                           # [B, 256, 1176]
        "input_ids": padded_inputs["input_ids"],                # [B, T]
        "attention_mask": padded_inputs["attention_mask"],      # [B, T]
        "labels": padded_labels,                                # [B, T] with -100
        "image_grid_thw": image_grid_thw                        # [B, 3]
    }


In [36]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files=csv_path, split="train", streaming=True)
dataset = dataset.filter(lambda x: x['image_path'] and x["question"] and x["answer"])

Let's take an overview look at the dataset. We shall see what the 3rd image is, and what caption it had.

In [37]:
vqa_dataset = VQALazyIterableDataset(dataset,lenght_of_dataset, tokenizer)


In [38]:
dataloader = DataLoader(
    vqa_dataset,
    batch_size=1,
)
batch = next(iter(dataloader))
for k, v in batch.items():
    print(k, v.shape)


pixel_values torch.Size([1, 256, 1176])
input_ids torch.Size([1, 300])
attention_mask torch.Size([1, 300])
labels torch.Size([1, 300])
image_grid_thw torch.Size([1, 3])


In [39]:
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig
from transformers import Trainer, TrainingArguments

FastVisionModel.for_training(model) # Enable for training!

trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = collate_fn, # Must use!
    train_dataset = vqa_dataset,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 10,
        save_steps=1000,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        report_to = "none",     # For Weights and Biases
        save_total_limit=2,
        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        max_steps = max_steps


    ),
)

/tmp/ipykernel_35/2149036836.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [32]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.494 GB of memory reserved.


In [40]:
# checkpoint_path = '/content/outputs/checkpoint-30'

trainer_stats = trainer.train(resume_from_checkpoint=checkpoint_path)


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,217,194 | Num Epochs = 2 | Total steps = 304,298
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 7,237,632/2,000,000,000 (0.36% trained)


Step,Training Loss
10,9.413400
20,9.341900
30,6.225300
40,6.668300
50,8.848500
60,6.560100
70,5.347400
80,6.111400
90,5.750500
100,5.506900


ZeroDivisionError: division by zero

In [ ]:
model.save_pretrained(f"{model_dir}/lora_model")  # Local saving
tokenizer.save_pretrained(f"{model_dir}/lora_model")
